In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ricardormotta/projects/LTV_analysis/ltv-ml-project/conf/local/gcp_token.json"


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.calibration import calibration_curve 


from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV


In [3]:
X_train = catalog.load("X_train")
X_test = catalog.load("X_test")
y_train = catalog.load("y_train_ltv")
y_test = catalog.load("y_test_ltv")
CT = catalog.load("CT")

[11/27/23 22:48:02] INFO     Loading data from 'X_train' (CSVDataSet)...                        ]8;id=921778;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=771874;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'X_test' (CSVDataSet)...                         ]8;id=652389;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=588040;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'y_train_ltv' (CSVDataSet)...                    ]8;id=816828;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=46766;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'y_test_ltv' (CSVDataSet)...                     ]8;id=341143;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=636867;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'CT' (PickleDataSet)...                          ]8;id=37342;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=777213;file:///home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [4]:
for X in [X_train, X_test]:
    X.loc[:,"days_to_churn"]=0

In [5]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
def regression_metrics(true_values, predicted_values, model_name):
    metrics = {
        'Mean Absolute Error (MAE)': mean_absolute_error(true_values, predicted_values),
        'Mean Squared Error (MSE)': mean_squared_error(true_values, predicted_values),
        'Root Mean Squared Error (RMSE)': mean_squared_error(true_values, predicted_values, squared=False),
        'R-squared': r2_score(true_values, predicted_values)
    }

    # Create a Pandas DataFrame from the metrics
    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=[model_name])
    
    return metrics_df


In [6]:
models = [
    RandomForestRegressor(),
    XGBRegressor(),
    GradientBoostingRegressor()
]
metrics = []
for model in models:
    pipe = Pipeline([
        ("CT", CT),
        ("model", model)
    ])
    pipe.fit(X_train, y_train)
    metrics.append(
        regression_metrics(y_train.values, pipe.predict(X_train), type(pipe["model"]).__name__+"_train")
    )
    metrics.append(
        regression_metrics(y_test.values, pipe.predict(X_test), type(pipe["model"]).__name__+"_test")
    )

                    WARNING  /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-pac ]8;id=9520;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=497211;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             kages/sklearn/base.py:1152: DataConversionWarning: A column-vector y                  
                             was passed when a 1d array was expected. Please change the shape of y                 
                             to (n_samples,), for example using ravel().                                           
                               return fit_method(estimator, *args, **kwargs)                                       
                                                                                                                   

[11/27/23 22:48:06] WARNING  /home/ricardormotta/projects/LTV_analysis/venv/lib/python3.10/site-pac ]8;id=973050;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=774653;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             kages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A                           
                             column-vector y was passed when a 1d array was expected. Please change                
                             the shape of y to (n_samples, ), for example using ravel().                           
                               y = column_or_1d(y, warn=True)                                                      
                                                                                                                   

In [7]:
pd.concat(metrics, axis=1)

,RandomForestRegressor_train,RandomForestRegressor_test,XGBRegressor_train,XGBRegressor_test,GradientBoostingRegressor_train,GradientBoostingRegressor_test
Mean Absolute Error (MAE),13.186830,13.135322,13.184090,13.134286,13.237728,13.147227
Mean Squared Error (MSE),257.918846,256.092356,257.899241,256.066801,259.066080,255.871837
Root Mean Squared Error (RMSE),16.059852,16.002886,16.059242,16.002087,16.095530,15.995994
R-squared,0.071042,0.064884,0.071113,0.064977,0.066910,0.065689


In [8]:
xgb = XGBRegressor()
parameters = {
    "n_estimators": [250, 500, 1000],
    "max_depth": [4, 6, 8]
}
clf = GridSearchCV(xgb, parameters, n_jobs=-1, verbose=3)
pipe = Pipeline([
    ("CT", CT),
    ("model", clf)
])
pipe.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['product', 'channel',
                                                   'age_bucket',
                                                   'operating_system']),
                                                 ('numerical', StandardScaler(),
                                                  ['days_to_churn', 'product_x',
                                                   'product_y'])])),
                ('model',
                 GridSearchCV(estimator=XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     co...
                                                     max_bin=None,
                                                     max_cat_threshold=None,
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...),
                              n_jobs=-1,
                              param_grid={'max_depth': [4, 6, 8],
                                          'n_estimators': [250, 500, 1000]},
                              verbose=3))])

In [9]:
regression_metrics(y_test.values, pipe.predict(X_test), type(pipe["model"]).__name__)

,GridSearchCV
Mean Absolute Error (MAE),13.129393
Mean Squared Error (MSE),255.787752
Root Mean Squared Error (RMSE),15.993366
R-squared,0.065996


In [10]:
pipe["model"].best_params_

{'max_depth': 4, 'n_estimators': 250}

In [11]:
y_test.mean()


months_as_client    22.383696
dtype: float64

[CV 1/5] END .....max_depth=4, n_estimators=500;, score=0.060 total time=   3.1s
[CV 3/5] END .....max_depth=6, n_estimators=250;, score=0.064 total time=   2.4s
[CV 5/5] END .....max_depth=6, n_estimators=500;, score=0.071 total time=   3.5s
[CV 2/5] END .....max_depth=8, n_estimators=500;, score=0.064 total time=   2.3s
[CV 2/5] END .....max_depth=4, n_estimators=500;, score=0.065 total time=   3.3s
[CV 5/5] END .....max_depth=6, n_estimators=250;, score=0.071 total time=   2.4s
[CV 1/5] END ....max_depth=6, n_estimators=1000;, score=0.060 total time=   5.6s
[CV 2/5] END ....max_depth=4, n_estimators=1000;, score=0.065 total time=   6.2s
[CV 4/5] END ....max_depth=6, n_estimators=1000;, score=0.066 total time=   4.8s
[CV 4/5] END .....max_depth=4, n_estimators=250;, score=0.066 total time=   1.6s
[CV 3/5] END ....max_depth=4, n_estimators=1000;, score=0.065 total time=   6.0s
[CV 3/5] END .....max_depth=8, n_estimators=250;, score=0.064 total time=   1.9s
[CV 4/5] END .....max_depth=